In [10]:
from ultralytics import solutions

solutions.ParkingPtsSelection()

requirements: Ultralytics requirement ['tkinter'] not found, attempting AutoUpdate...
Retry 1/2 failed: Command 'pip install --no-cache-dir "tkinter" ' returned non-zero exit status 1.
Retry 2/2 failed: Command 'pip install --no-cache-dir "tkinter" ' returned non-zero exit status 1.
requirements:  Command 'pip install --no-cache-dir "tkinter" ' returned non-zero exit status 1.


In [13]:
import cv2

# 車位邊框JSON
polygon_json_path = "bounding_boxes.json"

cap = cv2.VideoCapture("../video/Khare_testvideo_07.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter("parking management.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# 初始化停車管理系統
management = solutions.ParkingManagement(model_path="../models/v6.pt", margin=1)

while cap.isOpened():
    ret, im0 = cap.read()
    if not ret:
        break

    # 從JSON文件中提取車位的邊界框數據
    json_data = management.parking_regions_extraction(polygon_json_path)

    # 使用YOLOv8模型對當前幀進行車輛檢測和跟踪
    results = management.model.track(im0, persist=True, show=False)

    # 如果檢測到車輛，則提取檢測結果的邊界框和類別
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().tolist()    # 提取邊界框坐標
        clss = results[0].boxes.cls.cpu().tolist()  # 提取車輛類別
        management.process_data(json_data, im0, boxes, clss)    # 處理檢測結果並更新停車區域狀態

    management.display_frames(im0)  # 在當前幀上顯示檢測結果
    video_writer.write(im0) # 將當前幀寫入到輸出視頻文件

cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 42 cars, 1 person, 16.4ms
Speed: 0.0ms preprocess, 16.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 42 cars, 1 person, 15.5ms
Speed: 1.5ms preprocess, 15.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 42 cars, 1 person, 17.5ms
Speed: 0.0ms preprocess, 17.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 42 cars, 1 person, 16.0ms
Speed: 1.0ms preprocess, 16.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 42 cars, 1 person, 16.5ms
Speed: 1.0ms preprocess, 16.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 42 cars, 1 person, 16.0ms
Speed: 0.0ms preprocess, 16.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 42 cars, 1 person, 15.1ms
Speed: 0.5ms preprocess, 15.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 42 cars, 1 person, 15.0ms
Speed: 1.0ms prep